In [1]:
cd ..

/home/mushahid/Desktop/project/cpenprojec/UnsupGenModbyMPS


In [2]:
import jax.numpy as jnp
import numpy as np
import pennylane as qml
from pennylane import numpy as pnp
import random_near_identity_extended_circuit as random_near_identity_extended_circuit
from MPScumulant import MPS_c
import mps_circuit_helpers as helpers


In [3]:
results = {}

## Initial Weights: Either Randomly Assigned With No Restrictions or Weights Near Unitary

In [4]:
weights_random = jnp.array(pnp.random.normal(size=(88, 15), requires_grad=True))

weights_near_identity = jnp.array(pnp.random.uniform(low = -0.01, high=0.01, size=(88, 15), requires_grad=True))

params_random = {
    'params': weights_random
}
params_near_identity = {
    'params': weights_near_identity
}

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [5]:
random_initialized_circuit = random_near_identity_extended_circuit.get_random_initialized_circuit(weights_random)
near_unitary_initialized_circuit = random_near_identity_extended_circuit.get_random_initialized_circuit(weights_near_identity)

params_random, loss_track_random = random_near_identity_extended_circuit.train_model(params_random)
params_near_identity, loss_track_near_unitary = random_near_identity_extended_circuit.train_model(params_near_identity)

results['Random Initialization'] = [params_random, loss_track_random]
results['Near Identity Initialization'] = [params_near_identity, loss_track_near_unitary]


100%|██████████| 15000/15000 [20:29<00:00, 12.20it/s]


## Extending Circuit Generate by MPS

In [ ]:
chis = ['BStest/BS_project-2-MPS', 'BStest/BS_project-4-MPS', 'BStest/BS_project-8-MPS','BS_4_3-300-MPS']
labels = ["2 Max Bond Dimension MPS", "Chi = 4 Max Bond Dimension MPS", "Chi = 8 Max Bond Dimension MPS", "Chi = Max Bond Dimension 300 MPS"]
maxi_bonds = [2, 4, 8, 300]
for i in range(len(chis)):
    random_near_identity_extended_circuit.maxi_bond = maxi_bonds[i]
    random_near_identity_extended_circuit.chi = chis[i]
    random_near_identity_extended_circuit.mps = MPS_c(12, max_bond_dim=maxi_bonds[i])

    data_states = random_near_identity_extended_circuit.get_data_states("BStest/b_s_4_3.npy", 12)
    
    total_num_weights, wires_to_add_su4 = random_near_identity_extended_circuit.get_details_for_su4_matrices(expanded_tape, n_wires)
    expanded_tape = random_near_identity_extended_circuit.get_expanded_tape()
    params = jnp.array(qml.math.stack(expanded_tape.data))
    params1 = jnp.array(pnp.random.normal(size=(total_num_weights, 15), requires_grad=True))
    init_val = {
    'params': params,
    'params1': params1
    }      
    init_val, loss_track = random_near_identity_extended_circuit.train_model(init_val)
    results[labels[i]] = [init_val, loss_track]
